In [29]:
import pandas as pd
import numpy as np
from itertools import product

In [53]:
# Carga de los CSVs que contienen los datos con los que se van a trabajar

df_restaurantes = pd.read_csv("C:/Users/jesus/Prueba_tecnica_Deloitte/restaurantes.csv", sep=";")
df_publicidad = pd.read_csv("C:/Users/jesus/Prueba_tecnica_Deloitte/publicidad.csv", 
                            delimiter=";", header=None, names=["timestamp_publicidad", "lat2", "long2", "panfletos"])
df_ventas = pd.read_csv("C:/Users/jesus/Prueba_tecnica_Deloitte/ventas.csv", delimiter=";", header=None, names=["timestamp_venta", "id", "ingresos"])

df_restaurantes = df_restaurantes.rename(columns={"lat": "lat1", "long": "long1"})



In [54]:
df_publicidad.head()

,timestamp_publicidad,lat2,long2,panfletos
0,1420099740000,42.898177,-8.512549,2116
1,1420099800000,37.359507,-5.937782,253
2,1420099920000,41.414480,2.149441,1018
3,1420099980000,40.404021,-3.680018,16
4,1420100280000,41.308698,2.149767,1264


In [55]:
df_ventas.head()

,timestamp_venta,id,ingresos
0,1420066893000,57,160.90
1,1420067011000,97,111.67
2,1420067190000,38,59.66
3,1420067213000,71,243.20
4,1420067268000,72,94.59


In [56]:
df_restaurantes.head()

,punto,id,lat1,long1
0,01010000008F9911990F1F0140E390BF99FAAA4440,1,41.335773,2.140166
1,0101000000F71502CAB7E700401E3EA2CB72B64440,2,41.425378,2.113144
2,01010000003633BD643B0E0140C1ED277D08A64440,3,41.297134,2.131949
3,01010000003C83DF8E97070140C9514B06CCB54440,4,41.420289,2.128707
4,0101000000ECF871CFE8BE0040A0F4FF8D4DAA4440,5,41.330492,2.093217


In [57]:
# Elimina la columna 'punto' si existe en df_restaurantes
df_restaurantes = df_restaurantes.drop(columns=["punto"])

# Elimina las filas con id=0 en df_ventas
df_ventas = df_ventas[df_ventas["id"] != 0]

In [58]:
# Define los valores máximos empleados como criterio de filtrado de datos
distancia_max = 5  # en km
tiempo_max = 168  # en horas

In [59]:
# Función empleada para calcular la distancia Haversine entre dos pares de cordenadas
def haversine(lat1, long1, lat2, long2):
    R = 6371  # Radio de la Tierra en km
    lat1 = np.radians(lat1)
    lon1 = np.radians(long1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(long2)
    
    dlat = lat2 - lat1
    dlon = long2 - long1
    
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    return R * c

In [60]:
# Creación de todas las combinaciones de restaurantes y acciones publicitarias
combinaciones = pd.DataFrame(list(product(df_restaurantes.index, df_publicidad.index)), columns=["restaurante_idx", "publicidad_idx"])

In [61]:
combinaciones.head()

,restaurante_idx,publicidad_idx
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4


In [62]:
# Unir datos de restaurantes y publicidad a las combinaciones
combinaciones = combinaciones.merge(df_restaurantes, left_on="restaurante_idx", right_index=True)
combinaciones = combinaciones.merge(df_publicidad, left_on="publicidad_idx", right_index=True, suffixes=("_restaurante", "_publicidad"))

In [63]:
combinaciones.head()

,restaurante_idx,publicidad_idx,id,lat1,long1,timestamp_publicidad,lat2,long2,panfletos
0,0,0,1,41.335773,2.140166,1420099740000,42.898177,-8.512549,2116
1,0,1,1,41.335773,2.140166,1420099800000,37.359507,-5.937782,253
2,0,2,1,41.335773,2.140166,1420099920000,41.414480,2.149441,1018
3,0,3,1,41.335773,2.140166,1420099980000,40.404021,-3.680018,16
4,0,4,1,41.335773,2.140166,1420100280000,41.308698,2.149767,1264


In [64]:
# Calcula la distancia (km) entre cada restaurante y cada acción publicitaria
combinaciones["distancia"] = combinaciones.apply(lambda row: haversine(row["lat1"], row["long1"], row["lat2"], row["long2"]), axis=1)

In [65]:
combinaciones.head()

,restaurante_idx,publicidad_idx,id,lat1,long1,timestamp_publicidad,lat2,long2,panfletos,distancia
0,0,0,1,41.335773,2.140166,1420099740000,42.898177,-8.512549,2116,8301.026632
1,0,1,1,41.335773,2.140166,1420099800000,37.359507,-5.937782,253,8277.451223
2,0,2,1,41.335773,2.140166,1420099920000,41.414480,2.149441,1018,45.193805
3,0,3,1,41.335773,2.140166,1420099980000,40.404021,-3.680018,16,2224.320705
4,0,4,1,41.335773,2.140166,1420100280000,41.308698,2.149767,1264,46.035950


In [66]:
# Filtro combinaciones donde la distancia es menor o igual a 5 km
combinaciones_filtradas = combinaciones[combinaciones["distancia"] <= distancia_max]

In [67]:
combinaciones_filtradas.head()

,restaurante_idx,publicidad_idx,id,lat1,long1,timestamp_publicidad,lat2,long2,panfletos,distancia
280,0,280,1,41.335773,2.140166,1422087300000,41.346657,2.139955,134,1.577656
1143,0,1143,1,41.335773,2.140166,1428304440000,41.333432,2.139745,2138,2.033184
2039,0,2039,1,41.335773,2.140166,1434699900000,41.378460,2.140460,1868,4.950201
3287,1,1115,2,41.425378,2.113144,1428046020000,41.437017,2.114130,319,4.884480
3936,1,1764,2,41.425378,2.113144,1432797060000,41.415925,2.112343,1283,3.968365


In [68]:
# Une ventas con combinaciones filtradas (las ventas que ocurren a menos de 5 km de la acción publicitaria)
resultados = df_ventas.merge(combinaciones_filtradas, how="inner", left_on="id", right_on="id")

In [69]:
# Asegura de que los timestamps estén en formato datetime
resultados["timestamp_venta"] = pd.to_datetime(resultados["timestamp_venta"], unit="ms")
resultados["timestamp_publicidad"] = pd.to_datetime(resultados["timestamp_publicidad"], unit="ms")

In [70]:
# Calculo del tiempo en horas entre la acción publicitaria y la venta
resultados["tiempo"] = (resultados["timestamp_venta"] - resultados["timestamp_publicidad"]).dt.total_seconds() / 3600.0 

In [73]:
# Establece la condición donde el tiempo es menor o igual a 168 horas y mayor que cero
resultados_filtrados = resultados[(resultados["tiempo"] <= tiempo_max) & (resultados["tiempo"] >= 0)]

In [74]:
resultados_filtrados.head()

,timestamp_venta,id,ingresos,restaurante_idx,publicidad_idx,lat1,long1,timestamp_publicidad,lat2,long2,panfletos,distancia,tiempo
10308,2015-01-01 08:10:39,76,63.01,75,0,42.879157,-8.511682,2015-01-01 08:09:00,42.898177,-8.512549,2116,4.568876,0.027500
10579,2015-01-01 08:13:03,76,191.85,75,0,42.879157,-8.511682,2015-01-01 08:09:00,42.898177,-8.512549,2116,4.568876,0.067500
10598,2015-01-01 08:13:17,31,50.70,30,3,40.427912,-3.679913,2015-01-01 08:13:00,40.404021,-3.680018,16,2.704972,0.004722
10621,2015-01-01 08:13:24,58,256.77,57,1,37.381111,-5.938042,2015-01-01 08:10:00,37.359507,-5.937782,253,2.738580,0.056667
10639,2015-01-01 08:13:33,76,60.75,75,0,42.879157,-8.511682,2015-01-01 08:09:00,42.898177,-8.512549,2116,4.568876,0.075833


In [75]:
# Normalización de distancia y tiempo
resultados_filtrados["distancia_normalizada"] = resultados_filtrados["distancia"] / distancia_max
resultados_filtrados["tiempo_normalizada"] = resultados_filtrados["tiempo"] / tiempo_max

C:\Users\jesus\AppData\Local\Temp\ipykernel_21192\1799617113.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultados_filtrados["distancia_normalizada"] = resultados_filtrados["distancia"] / distancia_max
C:\Users\jesus\AppData\Local\Temp\ipykernel_21192\1799617113.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultados_filtrados["tiempo_normalizada"] = resultados_filtrados["tiempo"] / tiempo_max


In [76]:
# Calculo eficiencia para cada venta que cumple con las dos condiciones establecidas
resultados_filtrados["eficiencia"] = resultados_filtrados["panfletos"] / ((resultados_filtrados["distancia_normalizada"]**2 
                                                                           + resultados_filtrados["tiempo_normalizada"]**2))

C:\Users\jesus\AppData\Local\Temp\ipykernel_21192\217774453.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultados_filtrados["eficiencia"] = resultados_filtrados["panfletos"] / ((resultados_filtrados["distancia_normalizada"]**2


In [78]:
# Agrupar por id del restaurante y calcular el sumatorio de la eficiencia
sumatorio_eficiencia_por_restaurante = resultados_filtrados.groupby('id')['eficiencia'].sum().reset_index()
sumatorio_eficiencia_por_restaurante.columns = ['id', 'sumatorio_eficiencia']

In [79]:
# Unir el sumatorio de la eficiencia al DataFrame original
resultados_filtrados = resultados_filtrados.merge(sumatorio_eficiencia_por_restaurante, on='id', how='left')

In [82]:
# Normalizar la eficiencia y multiplicar por los ingresos
resultados_filtrados["ingresos_ajustados"] = (resultados_filtrados["eficiencia"] / 
                                              resultados_filtrados["sumatorio_eficiencia"]) * resultados_filtrados["ingresos"]

In [94]:
# Selección de las columnas requeridas para el DataFrame final
resultados_finales = resultados_filtrados[["lat2", "long2", "timestamp_publicidad", "ingresos_ajustados"]]

resultados_finales.columns = ["latitud_publicidad", "longitud_publicidad", "timestamp_publicidad", "eficiencia"]

In [95]:
resultados_finales.head()

,latitud_publicidad,longitud_publicidad,timestamp_publicidad,eficiencia
0,42.898177,-8.512549,2015-01-01 08:09:00,0.014580
1,42.898177,-8.512549,2015-01-01 08:09:00,0.044394
2,40.404021,-3.680018,2015-01-01 08:13:00,0.001012
3,37.359507,-5.937782,2015-01-01 08:10:00,0.032004
4,42.898177,-8.512549,2015-01-01 08:09:00,0.014058


In [99]:
# Guardar el DataFrame final como un archivo CSV
resultados_finales.to_csv("C:/Users/jesus/Prueba_tecnica_Deloitte/prueba_tecnica_Deloitte.csv", index=False)